<a href="https://colab.research.google.com/github/vhppacheco/ic-ita-finance-ia/blob/main/data/DFs_Economatica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Bibliotecas

In [ ]:
!pip install -q tsfeatures

In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

#Função de Construção do DataFrame

### A função ***build_df*** tem por objetivo realizar um processo de construção de DataFrames dos **índices Ibovespa e S&P500** através de Screenings e Matrizes constuídas na Plataforma **Economatica**
### Seu único parâmetro é uma *string* ("Ibov" ou "SP500")

In [ ]:
def build_df(indice):

  #Screening
  scr = pd.read_excel(f'/content/drive/MyDrive/IC ITA/Projeto de Pesquisa - IC ITA/Economatica/Bases de Dados/Screening_{indice}.xlsx', skiprows = 3)
  codigos= scr['Código']

  #Indicadores
  df0 = pd.read_excel(f'/content/drive/MyDrive/IC ITA/Projeto de Pesquisa - IC ITA/Economatica/Bases de Dados/Matrixx_Indicadores_{indice}.xlsx', sheet_name= None, skiprows = 3)

  df = pd.DataFrame()

  for cod in codigos:
    df_temp = df0[cod]
    print(cod)
    df_temp['Código'] = cod
    df = df.replace('-', np.nan).fillna(method='ffill')
    df = df.fillna(method='bfill')
    df = df.append(df_temp)
    # df = df.replace('-', np.nan).fillna(method='bfill')
    # df = df.fillna(method='ffill')

  #Screening + Indicadores
  df = pd.merge(df,scr, how = 'left', on = 'Código').set_index('Data')
  df.columns = df.columns.str.replace('\\n',' ', regex=True)
  df = df.drop(df.iloc[:,23:38], axis = 1)

  #Cotação Média Trimestral
  cotacao0= pd.read_excel(f'/content/drive/MyDrive/IC ITA/Projeto de Pesquisa - IC ITA/Economatica/Bases de Dados/Matrixx_Cotacao_{indice}.xlsx', sheet_name= None, skiprows = 3)

  cotacao_temp0 = pd.DataFrame()

  for cod in codigos:
    cotacao_temp1 = cotacao0[cod]
    print(cod)
    cotacao_temp1['Código'] = cod
    cotacao1 = cotacao_temp1[['Beta Ibov', 'Beta sp500',
       'TIR\nmédia trimestral\nem 3 meses\nEm moeda orig',
       'Bollinger\n90/2\nMédia\nEm moeda orig',
       'Volatilidade\nbase trimestral\n3 meses\nEm moeda orig', 'Sharpe Ibov',
       'Sharpe SP500',
       'VAR %\nbase diária\nconfiança: 95%\n3 meses\nEm moeda orig',
       'Max Drd (c/rec)\najust p/ prov\n3 meses\nEm moeda orig']].replace('-', np.nan).fillna(method='ffill')

    cotacao1 = cotacao1.fillna(method='bfill')

    cotacao_temp = cotacao_temp1[['Data','Media\ndo fechamento\nem 1 trimestre\nEm US Dollars\najust p/ prov','Fechamento\najust p/ prov\nEm moeda orig',' Data do Bal\n consolid:sim*','Código']]

    cotacao_temp[' Data do Bal\n consolid:sim*'].iloc[:1] = cotacao_temp['Data'].iloc[:1]
    cotacao_temp['Fechamento\najust p/ prov\nEm moeda orig'] = cotacao_temp['Fechamento\najust p/ prov\nEm moeda orig'].replace('-', np.nan)
    cotacao_temp['Fechamento\najust p/ prov\nEm moeda orig'] = cotacao_temp['Fechamento\najust p/ prov\nEm moeda orig'].fillna(method='ffill')
    cotacao_temp['Fechamento\najust p/ prov\nEm moeda orig _temp'] = cotacao_temp['Fechamento\najust p/ prov\nEm moeda orig'].fillna(method='bfill').iloc[:1]
    cotacao_temp['Fechamento\najust p/ prov\nEm moeda orig'].iloc[:1] = cotacao_temp['Fechamento\najust p/ prov\nEm moeda orig _temp'].iloc[:1]
    cotacao_temp = cotacao_temp.replace("-",np.nan)

    cotacao_temp2 = pd.concat([cotacao_temp,cotacao1], axis=1)
    cotacao_temp2 = cotacao_temp2[cotacao_temp2[' Data do Bal\n consolid:sim*'].notnull()]

    cotacao_temp2['Log Retornos Trimestral'] = (cotacao_temp2['Fechamento\najust p/ prov\nEm moeda orig'].pct_change(axis=0)).apply(np.log1p)
    cotacao_temp2 = cotacao_temp2[cotacao_temp2['Log Retornos Trimestral'].notnull()]

    cotacao_temp0 = cotacao_temp0.append(cotacao_temp2)

  #DataFrame de Datas + Filtro
  cotacao_temp0['Qdate'] = [date - pd.tseries.offsets.DateOffset(days=1) + pd.tseries.offsets.QuarterEnd() for date in  cotacao_temp0.Data]
  datas = cotacao_temp0['Qdate'].unique()
  datas = np.sort(datas)

  count_release = cotacao_temp0['Código'].groupby(cotacao_temp0['Código']).count().sort_values(ascending=False)
  count_release = pd.DataFrame(count_release)
  print("VER AQUI",count_release)
  count_release = count_release[(count_release['Código'] == 24)]

  trimestre = df.index.unique()

  df_datas = list(zip(datas, trimestre))
  df_datas = pd.DataFrame(df_datas, columns=['Data', 'Trimestres'])

  #Log Retornos Trimestrais + Datas
  log_returns = cotacao_temp0[cotacao_temp0.Código.isin(count_release.index)]
  log_returns = log_returns.fillna(method='ffill')
  log_returns = log_returns.set_index('Data')
  log_returns = pd.merge(df_datas, log_returns, how='inner', left_on = 'Data', right_on= 'Qdate').set_index('Trimestres')

  #DataFrame Final
  log = log_returns.reset_index()
  df_final = df.reset_index()
  log = log[['Trimestres','Log Retornos Trimestral','Beta Ibov','Beta sp500','TIR\nmédia trimestral\nem 3 meses\nEm moeda orig','Bollinger\n90/2\nMédia\nEm moeda orig','Volatilidade\nbase trimestral\n3 meses\nEm moeda orig', 'Sharpe Ibov','Sharpe SP500','VAR %\nbase diária\nconfiança: 95%\n3 meses\nEm moeda orig','Max Drd (c/rec)\najust p/ prov\n3 meses\nEm moeda orig',' Data do Bal\n consolid:sim*','Código']]
  df_final = pd.merge(df_final,log, how= 'inner', left_on= ['Código','Data'], right_on = ['Código','Trimestres']).set_index('Data')
  df_final.columns = df_final.columns.str.replace('\\n',' ', regex=True)
  df_final = df_final.replace('-', np.NaN).fillna(method='bfill')
  df_final = df_final.replace('-', np.NaN).fillna(method='ffill')
  df_final = df_final.sort_values(by=['Código',' Data do Bal  consolid:sim*'], ascending = True)
  df_final = df_final.drop(['Beta 3 tri Em US Dollars',
       'Media nulos = 0 do volume$ em 1 trimestre Em US Dollars em milhares',
       'TIR média trimestral em 1 trimestre Em US Dollars',
       'Bollinger 1/2 Média Em US Dollars',
       'Volatilidade base trimestral 1 anos Em US Dollars',
       'Correlacao 24 tri Em US Dollars', 'Sharpe 24 tri Em US Dollars','Unnamed: 0', 'Trimestres'], axis = 1)

  return df_final

# Ibovespa

In [ ]:
df_ibov = build_df(indice='Ibov')

In [ ]:
df_ibov

In [ ]:
df_ibov.columns

In [ ]:
s = df_ibov.pop('Código')
df_ibov = pd.concat([df_ibov, s], 1)
df_ibov = df_ibov.drop(['Beta sp500','Sharpe SP500'], axis = 1)
df_ibov

In [ ]:
df_ibov.to_excel('df_Ibov_Final.xlsx')

### Log Retornos Médio Trimestrais

In [ ]:
df_ibov_log = df_ibov[['Media do fechamento em 1 trimestre Em US Dollars ajust p/ prov','Código']]
df_ibov_log = df_ibov.pivot_table(index=df_ibov.index, columns='Código',values= 'Media do fechamento em 1 trimestre Em US Dollars ajust p/ prov', aggfunc='first')
df_ibov_log = (df_ibov_log.pct_change(axis=0)).apply(np.log1p)
df_ibov_log= df_ibov_log.iloc[1:,:]
df_ibov_log

In [ ]:
df_nan = pd.DataFrame(df_ibov_log.isna().sum())
df_nan

In [ ]:
plt.rcParams.update({'font.size': 15})
plt.rc('legend', fontsize=7.45) 
df_ibov_log.plot(figsize=(30,17))
plt.ylabel('Retornos Logarítmicos')
plt.xlabel('Data')
plt.show()

In [ ]:
df_ibov_log.plot.hist(bins=100,figsize=(20,17));

### Reprocesing - Ts Features

In [ ]:
from tsfeatures import tsfeatures as tf

In [ ]:
df_ibov_log.index.name = 'Data'
df_ts = df_ibov_log.reset_index().melt('Data', var_name=['unique_id'])
df_ts = df_ts.rename(columns={"Data":"ds","value":"y"})
df_ts = df_ts[['unique_id','ds','y']]
df_ts

In [ ]:
df_ts = df_ts[np.isfinite(df_ts['y']) == True]

In [ ]:
from pandas.core.arrays.numeric import T
df_features = tf(df_ts, freq=7)
df_features

In [ ]:
df_features = df_features.rename(columns={"unique_id":"Código"})
df_features.columns

In [ ]:
df_ibov_features = df_features.fillna(0)
df_ibov_features

In [ ]:
df_ibov_features.to_excel('df_Features_Ibov.xlsx')

# S&P500

In [ ]:
df_sp = build_df(indice='SP500')

In [ ]:
df_sp

In [ ]:
sp = df_sp.pop('Código')
df_sp = pd.concat([df_sp, sp], 1)
df_sp = df_sp.drop(['Beta Ibov','Sharpe Ibov'], axis = 1)
df_sp

In [ ]:
tri = ['1T2016','2T2016','3T2016','4T2016','1T2017','2T2017','3T2017','4T2017','1T2018','2T2018','3T2018','4T2018','1T2019','2T2019','3T2019','4T2019','1T2020','2T2020','3T2020','4T2020','1T2021','2T2021','3T2021','4T2021']

tempList = list(tri)

for i in range(464):
    for element in tempList:
        tri.append(element)

df_sp['Data'] = tri
df_sp = df_sp.set_index('Data')
df_sp

In [ ]:
df_sp.info()

#### Caso de Empresa com Balaços não enquadrados no trimestre

In [ ]:
df_aap = df_sp[df_sp['Código']=='AAP']
df_aap

In [ ]:
df_sp.to_excel('df_SP500_Final.xlsx')

### Log Retornos Médio Trimestrais

In [ ]:
df_sp_log = df_sp[['Media do fechamento em 1 trimestre Em US Dollars ajust p/ prov','Código']]
df_sp_log = df_sp.pivot_table(index=df_sp.index, columns='Código',values='Media do fechamento em 1 trimestre Em US Dollars ajust p/ prov', aggfunc='first')
df_sp_log = (df_sp_log.pct_change(axis=0)).apply(np.log1p)
df_sp_log= df_sp_log.iloc[1:,:]
df_sp_log

In [ ]:
df_nan = pd.DataFrame(df_sp_log.isna().sum())
df_nan_sum = df_nan[df_nan[0]>0]
df_nan_sum

In [ ]:
plt.rcParams.update({'font.size': 15})
plt.rc('legend', fontsize=7.45) 
df_sp_log.plot(figsize=(30,17))
plt.ylabel('Retornos Logarítmicos')
plt.xlabel('Data')
plt.show()

In [ ]:
df_sp_log.plot.hist(bins=100,figsize=(20,17));

### Reprocesing - Ts Features

In [ ]:
df_sp_log.index.name = 'Data'
df_ts = df_sp_log.reset_index().melt('Data', var_name=['unique_id'])
df_ts = df_ts.rename(columns={"Data":"ds","value":"y"})
df_ts = df_ts[['unique_id','ds','y']]
df_ts

In [ ]:
df_ts = df_ts[np.isfinite(df_ts['y']) == True]

In [ ]:
from pandas.core.arrays.numeric import T
df_features = tf(df_ts, freq=7)
df_features

In [ ]:
df_sp_features = df_features.fillna(0)
df_sp_features

In [ ]:
df_sp_features.to_excel('df_Features_SP500.xlsx')